#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2020


# Homework 1:  Information Retrieval Basics

### 100 points [7% of your final grade]

### Due: 

*Goals of this homework:* In this homework you will get first hand experience building a text-based mini search engine. In particular, there are three main learning objectives: (i) the basics of tokenization (e.g. stemming, case-folding, etc.) and its effect on information retrieval; (ii) basics of index building and Boolean retrieval; and (iii) basics of the Vector Space model and ranked retrieval.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, my homework submission would be something like `555001234_hw1.ipynb`. Submit this notebook via eCampus (look for the homework 1 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

## Dataset

The dataset is collected from Quizlet (https://quizlet.com), a website where users can generated their own flashcards. Each flashcard generated by a user is made up of an entity on the front and a definition describing or explaining the entity correspondingly on the back. We treat entities on each flashcard's front as the queries and the definitions on the back of flashcards as the documents. Definitions (documents) are relevant to an entity (query) if the definitions are from the back of the entity's flashcard; otherwise definitions are not relevant. **In this homework, queries and entities are interchangeable as well as documents and definitions.**

The format of the dataset is like this:

**query \t document id \t document**

Examples:

decision tree	\t 27946 \t	show complex processes with multiple decision rules.  display decision logic (if statements) as set of (nodes) questions and branches (answers).

where "decision tree" is the entity in the front of a flashcard and "show complex processes with multiple decision rules.  display decision logic (if statements) as set of (nodes) questions and branches (answers)." is the definition on the flashcard's back and "27946" is the id of the definition. Naturally, this document is relevant to the query.

false positive rate	\t 686	\t fall-out; probability of a false alarm

where document 686 is not relevant to query "decision tree" because the entity of "fall-out; probability of a false alarm" is "false positive rate".

# Part 1: Parsing (20 points)

First, you should tokenize documents (definitions) using **whitespaces and punctuations as delimiters**. Your parser needs to also provide the following three pre-processing options:
* Remove stop words: use nltk stop words list (from nltk.corpus import stopwords)
* Stemming: use [nltk Porter stemmer](http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.porter)
* Remove any other strings that you think are less informative or nosiy.

Please note that you should stick to the stemming package listed above. Otherwise, given the same query, the results generated by your code can be different from others.

In [1]:
# configuration options
remove_stopwords = True  # or false
use_stemming = True # or false
remove_otherNoise = True # or false

In [2]:
# Your parser function here. It will take the three option variables above as the parameters
# add cells as needed to organize your code

import re
import numpy as np
import pandas as pd
import math

from nltk.corpus import stopwords
from nltk.stem.porter import *
from pandas import DataFrame, Series
from collections import defaultdict
from sklearn.preprocessing import normalize


stop_words = set(stopwords.words('english'))

# load data
filename = "homework_1_data.txt"
with open(filename) as f:
    content = f.readlines()

# split data into 3 columns and convert it to a dataframe
documents = np.array([re.split("\t", x) for x in content])
documents_df = pd.DataFrame(documents, columns=["entity", "id", "definition"])        

In [3]:
def count_unique_words(definitions: Series):
    distinct_set = set()
    
    for index, value in definitions.items():
        distinct_set.update(value)
    
    return len(distinct_set)

def parse(definitions: Series, remove_stopwords: bool = True, 
          use_stemming: bool = True, remove_otherNoise: bool = True,
         show_log = True):
    definitions = definitions.apply(lambda x: re.split("\W+", x.strip()))
    
    if show_log:        
        print("None of pre-processing options =", count_unique_words(definitions))
    
    # remove stopwords
    if remove_stopwords:
        def rm_stopwords(tokens):
            removed = []
            for w in tokens:
                if w not in stop_words:
                    removed.append(w)
            return removed
        
        definitions = definitions.apply(rm_stopwords)
        if show_log:
            print("After removing stop words =", count_unique_words(definitions))

    # do stemming
    if use_stemming:
        stemmer = PorterStemmer()
        
        def stem(tokens):
            return [stemmer.stem(token) for token in tokens]
        
        definitions = definitions.apply(stem)
        if show_log:
            print("After removing stop words + stemming =", count_unique_words(definitions))
        
    # remove noise        
    if remove_otherNoise:
        def rm_noise(tokens):
            removed = []
            # to lower case, then remove digits and empty strings
            for w in tokens:
                w = w.lower()
                if not w.isdigit() and w.strip():
                    removed.append(w)
            return removed
                
        definitions = definitions.apply(rm_noise)
        if show_log:
            print("After removing stop words + stemming + removing other noise = %i \n" % count_unique_words(definitions))
    
    return definitions

In [4]:
# parse definitions
documents_df = documents_df.assign(tokens=parse(documents_df['definition']))


None of pre-processing options = 15939
After removing stop words = 15799
After removing stop words + stemming = 9921
After removing stop words + stemming + removing other noise = 9654 



In [5]:
# show parsed documents dataframe
documents_df

,entity,id,definition,tokens
0,bottom-up approach,0,estimates of duration and cost are made for ea...,"[estim, durat, cost, made, compon, separ, comb..."
1,bottom-up approach,1,pattern from process deductive approach use ...,"[pattern, process, deduct, approach, use, unde..."
2,bottom-up approach,2,normalization; 3nf\n,"[normal, 3nf]"
3,bottom-up approach,3,client factor standpoint\n,"[client, factor, standpoint]"
4,bottom-up approach,4,"approach for dynamic programming problems, whi...","[approach, dynam, program, problem, problem, s..."
...,...,...,...,...
30912,sensitivity analysis,30912,understand changes in the analysis outcome if ...,"[understand, chang, analysi, outcom, case, ass..."
30913,memory controller,30913,the chip that manages access to the ram and a ...,"[chip, manag, access, ram, other, comm, direct..."
30914,memory controller,30914,chip that manages access to ram.\n,"[chip, manag, access, ram]"
30915,memory controller,30915,this handles dram accesses by converting reque...,"[handl, dram, access, convert, request, proces..."


### Observations

Once you have your parser working, you should report here the size of your dictionary under the four cases. That is, how many unique tokens do you have with stemming on and casefolding on? And so on. You should fill in the following

* None of pre-processing options      = ??
* remove stop words       = ??
* remove stop words + stemming       = ??
* remove stop words + stemming  + remove other noise     = ??

#### My Answer

None of pre-processing options = 15939

After removing stop words = 15799

After removing stop words + stemming = 9921

After removing stop words + stemming + removing other noise = 9654 

# Part 2: Boolean Retrieval (30 points)

In this part you build an inverted index to support Boolean retrieval. We only require your index to support AND queries. In other words, your index does not have to support OR, NOT, or parentheses. Also, we do not explicitly expect to see AND in queries, e.g., when we query **relational model**, your search engine should treat it as **relational** AND **model**.

Search for the queries below using your index and print out matching documents (for each query, print out 5 matching documents):
* relational database
* garbage collection
* retrieval model

Please use the following format to present your results:
* query: relational database
* result 1:
* entity: database management system
* definition id: 656
* definition: software system used to manage databases
* result 2:
* ......
* query: garbage collection
* ......
* query: retrieval model
* ......

In [6]:
# build the index here
# add cells as needed to organize your code

inverted_indices = defaultdict(list)
for index, row in documents_df.iterrows():
    for token in set(row['tokens']):
        inverted_indices[token].append(index)

In [7]:
# search for the input using your index and print out ids of matching documents.
def intersect(query, documents_df, inverted_indices):
    br_2d_query = parse(Series(re.split(" ", query)), show_log=False)
    ans = []
    d1 = inverted_indices[br_2d_query[0][0]]
    d2 = inverted_indices[br_2d_query[1][0]]
    
    if not (d1 and d2):
        return DataFrame()
    
    len1, len2 = len(d1), len(d2)
    p1, p2 = 0, 0
    
    while p1 < len1 and p2 < len2:
        if d1[p1] == d2[p2]:
            ans.append(documents_df.iloc[d1[p1]])
            p1 += 1
            p2 += 1
        elif d1[p1] < d2[p2]:
            p1 += 1
        else:
            p2 += 1
    return DataFrame(ans)

def print_result(query, intersect_res, documents_df):
    print("query:", query)
    
    for i in range(0, min(len(intersect_res), 5)):
        print("result %i:" % (i+1))
        print("entity: %s" % intersect_res.iloc[i]["entity"])
        print("definition id: %s" % intersect_res.iloc[i]["id"])
        print("definition: %s" % intersect_res.iloc[i]["definition"])


In [8]:
query1 = "relational database"
query2 = "garbage collection"
query3 = "retrieval model"

print()
results1 = intersect(query1, documents_df, inverted_indices)
print_result(query1, results1, documents_df)
results1[:5]


query: relational database
result 1:
entity: database management system
definition id: 654
definition: dbms allows users to create, read, update, and delete structured data in a relational database. managers send requests to dbms and the dbms performs manipulation of the data. can retrieve information from using sql or qbe (query by example).   relational database management system: allows users to create, read, update, and delete data in a relational database.  pros: increased flexibility, inc scalability and performance, reduced info redundancy, inc info integrity/quality, increased info security.

result 2:
entity: database management system
definition id: 657
definition: general hospital utilizes various related files that include clinical and financial data to generate reports such as ms drg case mix reports. what application would be most effective for this activity  desktop publishing  word processing database management system command interpreter

result 3:
entity: database ma

,entity,id,definition,tokens
654,database management system,654,"dbms allows users to create, read, update, and...","[dbm, allow, user, creat, read, updat, delet, ..."
657,database management system,657,general hospital utilizes various related file...,"[gener, hospit, util, variou, relat, file, inc..."
682,database management system,682,database management system software that contr...,"[databas, manag, system, softwar, control, rel..."
741,relational model,741,"developed by ef codd of ibm in 19070, the rela...","[develop, ef, codd, ibm, relat, model, base, m..."
750,relational model,750,all relational database dbms products are buil...,"[relat, databas, dbm, product, built, model, e..."


In [9]:
results2 = intersect(query2, documents_df, inverted_indices)
print_result(query2, results2, documents_df)
results2[:5]

query: garbage collection
result 1:
entity: garbage collector
definition id: 4150
definition: the part of the operating system that performs garbage collection.

result 2:
entity: memory safety
definition id: 13115
definition: memory management handled differently such that there is garbage collection, prevent dangling pointer references.

result 3:
entity: garbage collection
definition id: 21550
definition: automatic memory management is made possible by garbage collection in .net framework. when a class object is created at runtime, certain memory space is allocated to it in the heap memory. however, after all the actions related to the object are completed in the program, the memory space allocated to it is a waste as it cannot be used. in this case, garbage collection is very useful as it automatically releases the memory space after it is no longer required.

result 4:
entity: garbage collection
definition id: 21553
definition: garbage collection (gc) is a form of automatic memory

,entity,id,definition,tokens
4150,garbage collector,4150,the part of the operating system that performs...,"[part, oper, system, perform, garbag, collect]"
13115,memory safety,13115,memory management handled differently such tha...,"[memori, manag, handl, differ, garbag, collect..."
21550,garbage collection,21550,automatic memory management is made possible b...,"[automat, memori, manag, made, possibl, garbag..."
21553,garbage collection,21553,garbage collection (gc) is a form of automatic...,"[garbag, collect, gc, form, automat, memori, m..."
21554,garbage collection,21554,there is a third phase of 2pc called garbage c...,"[third, phase, 2pc, call, garbag, collect, rep..."


In [10]:
results3 = intersect(query3, documents_df, inverted_indices)
print_result(query3, results3, documents_df)
results3[:5]

query: retrieval model
result 1:
entity: data model
definition id: 6983
definition: - a collection of concepts that can be used to describe the structure of a database - describes the structure - dbms is based on a data model   structure of data model:  - records - types - relationships - constraints - basic operations (specifying retrievals and updates)  types of data models: - high-level (conceptual i.e. er) - low level (physical i.e. xml) - implementation (representational) combines conceptual and physical i.e. relational model - nosql data models i.e. column, key-value, document stores

result 2:
entity: data model
definition id: 7031
definition: \ structure of data model:  - records - types - relationships - constraints - basic operations (specifying retrievals and updates)

result 3:
entity: query language
definition id: 9085
definition: used to update and retrieve data that is stored in a data model

result 4:
entity: physical design
definition id: 10327
definition: translate lo

,entity,id,definition,tokens
6983,data model,6983,- a collection of concepts that can be used to...,"[collect, concept, use, describ, structur, dat..."
7031,data model,7031,\ structure of data model: - records - types ...,"[structur, data, model, record, type, relation..."
9085,query language,9085,used to update and retrieve data that is store...,"[use, updat, retriev, data, store, data, model]"
10327,physical design,10327,translate logical model into technical specifi...,"[translat, logic, model, technic, specif, stor..."
10335,physical design,10335,translate logical model into technical specifi...,"[translat, logic, model, technic, specif, stor..."


### Observations
Could your boolean search engine find relevant documents for these queries? What is the impact of the three pre-processing options? Do they improve your search quality?

#### My Answer:

Yes.

The first and third option will only improve very little for the querying speed. The second option will improve the search quality.

# Part 3: Ranking Documents (50 points) 

In this part, your job is to rank the documents that have been retrieved by the Boolean Retrieval component in Part 2, according to their relevance with each query.

### A: Ranking with simple sums of TF-IDF scores (15 points) 
For a multi-word query, we rank documents by a simple sum of the TF-IDF scores for the query terms in the document.
TF is the log-weighted term frequency $1+log(tf)$; and IDF is the log-weighted inverse document frequency $log(\frac{N}{df})$

**Output:**
For each given query in Part 2, you should just rank the documents retrieved by your boolean search. You only need to output the top-5 results plus the TF-IDF sum score of each of these documents. Please use the following format to present your results:

* query: relational database
* result 1:
* score: 0.1
* entity: database management system
* definition id: 656
* definition: software system used to manage databases
* result 2:
* ......
* query: garbage collection
* ......
* query: retrieval model
* ......

In [11]:
# your code here
# hint: you could first call boolean retrieval function in part 2 to find possible relevant documents, 
# and then rank these documents in this part. Hence, you don't need to rank all documents.

def tf_idf_cal(query, results, documents, N):
    br_2d_query = parse(Series(re.split(" ", query)), show_log=False)
    
    df1 = documents_df["tokens"].apply(lambda tokens: 1 if br_2d_query[0][0] in tokens else 0).sum()
    df2 = documents_df["tokens"].apply(lambda tokens: 1 if br_2d_query[1][0] in tokens else 0).sum()
    
    # calculate TF-IDF score
    score1 = results["tokens"].apply(lambda x: (1 + math.log10(x.count(br_2d_query[0][0]))) * math.log10(N / df1))
    score2 = results["tokens"].apply(lambda x: (1 + math.log10(x.count(br_2d_query[1][0]))) * math.log10(N / df2))
    
    # sum up the scores of two terms
    score = score1 + score2
    
    return score

def print_scored_result(query, score_results, score_label, documents_df):
    print("query:", query)
    
    for i in range(0, min(len(score_results), 5)):
        print("result %i:" % (i+1))
        print("score: %f" % score_results.iloc[i][score_label])
        print("entity: %s" % score_results.iloc[i]["entity"])
        print("definition id: %s" % score_results.iloc[i]["id"])
        print("definition: %s" % score_results.iloc[i]["definition"])

In [12]:
N = len(documents)
score_label = "score"

results1 = results1.assign(score=tf_idf_cal(query1, results1, documents_df, N))
sorted_tf_idf_results1 = results1.sort_values(score_label, ascending=False)
print_scored_result(query1, sorted_tf_idf_results1, score_label, documents_df)
sorted_tf_idf_results1[:5]

query: relational database
result 1:
score: 4.717339
entity: relational algebra
definition id: 7156
definition: - a theoretical language with operations that work on one or more relations to define another relation without changing the original relation(s)  - relation-at-a-time (or set) language in which all tuples, possibly from several relations, are manipulated in one statement without looping  relational algebra, first created by edgar f. codd while at ibm, is a family of algebras with a well-founded semantics used for modelling the data stored in relational databases, and defining queries on it.  the main application of relational algebra is providing a theoretical foundation for relational databases, particularly query languages for such databases, chief among which is sql.

result 2:
score: 4.357658
entity: relational database
definition id: 28378
definition: a type of database system where data is stored in  tables related by common fields. a relational database is the most com

,entity,id,definition,tokens,score
7156,relational algebra,7156,- a theoretical language with operations that ...,"[theoret, languag, oper, work, one, relat, def...",4.717339
28378,relational database,28378,a type of database system where data is stored...,"[type, databas, system, data, store, tabl, rel...",4.357658
28254,relational database,28254,finite set of relations​. each relation consis...,"[finit, set, relat, relat, consist, schema, in...",4.238365
741,relational model,741,"developed by ef codd of ibm in 19070, the rela...","[develop, ef, codd, ibm, relat, model, base, m...",4.122275
654,database management system,654,"dbms allows users to create, read, update, and...","[dbm, allow, user, creat, read, updat, delet, ...",4.017821


In [13]:
results2 = results2.assign(score=tf_idf_cal(query2, results2, documents_df, N))
sorted_tf_idf_results2 = results2.sort_values(score_label, ascending=False)
print_scored_result(query2, sorted_tf_idf_results2, score_label, documents_df)
sorted_tf_idf_results2[:5]

query: garbage collection
result 1:
score: 6.530032
entity: garbage collection
definition id: 21553
definition: garbage collection (gc) is a form of automatic memory management. the garbage collector, or just collector, attempts to reclaim garbage, or memory occupied by objects that are no longer in use by the program.

result 2:
score: 6.329230
entity: garbage collection
definition id: 21550
definition: automatic memory management is made possible by garbage collection in .net framework. when a class object is created at runtime, certain memory space is allocated to it in the heap memory. however, after all the actions related to the object are completed in the program, the memory space allocated to it is a waste as it cannot be used. in this case, garbage collection is very useful as it automatically releases the memory space after it is no longer required.

result 3:
score: 6.329230
entity: garbage collection
definition id: 21554
definition: there is a third phase of 2pc called garb

,entity,id,definition,tokens,score
21553,garbage collection,21553,garbage collection (gc) is a form of automatic...,"[garbag, collect, gc, form, automat, memori, m...",6.530032
21550,garbage collection,21550,automatic memory management is made possible b...,"[automat, memori, manag, made, possibl, garbag...",6.329230
21554,garbage collection,21554,there is a third phase of 2pc called garbage c...,"[third, phase, 2pc, call, garbag, collect, rep...",6.329230
21559,garbage collection,21559,garbage collection is a feature that automatic...,"[garbag, collect, featur, automat, delet, unus...",5.520629
4150,garbage collector,4150,the part of the operating system that performs...,"[part, oper, system, perform, garbag, collect]",4.864784


In [14]:
results3 = results3.assign(score=tf_idf_cal(query3, results3, documents_df, N))
sorted_tf_idf_results3 = results3.sort_values(score_label, ascending=False)
print_scored_result(query3, sorted_tf_idf_results3, score_label, documents_df)
sorted_tf_idf_results3[:5]

query: retrieval model
result 1:
score: 4.292305
entity: data model
definition id: 6983
definition: - a collection of concepts that can be used to describe the structure of a database - describes the structure - dbms is based on a data model   structure of data model:  - records - types - relationships - constraints - basic operations (specifying retrievals and updates)  types of data models: - high-level (conceptual i.e. er) - low level (physical i.e. xml) - implementation (representational) combines conceptual and physical i.e. relational model - nosql data models i.e. column, key-value, document stores

result 2:
score: 3.337042
entity: data model
definition id: 7031
definition: \ structure of data model:  - records - types - relationships - constraints - basic operations (specifying retrievals and updates)

result 3:
score: 3.337042
entity: query language
definition id: 9085
definition: used to update and retrieve data that is stored in a data model

result 4:
score: 3.337042
entit

,entity,id,definition,tokens,score
6983,data model,6983,- a collection of concepts that can be used to...,"[collect, concept, use, describ, structur, dat...",4.292305
7031,data model,7031,\ structure of data model: - records - types ...,"[structur, data, model, record, type, relation...",3.337042
9085,query language,9085,used to update and retrieve data that is store...,"[use, updat, retriev, data, store, data, model]",3.337042
10327,physical design,10327,translate logical model into technical specifi...,"[translat, logic, model, technic, specif, stor...",3.337042
10335,physical design,10335,translate logical model into technical specifi...,"[translat, logic, model, technic, specif, stor...",3.337042


### B: Ranking with vector space model with TF-IDF (15 points) 

**Cosine:** You should use cosine as your scoring function. 

**TFIDF:** For the document vectors, use the standard TF-IDF scores as introduced in A. For the query vector, use simple weights (the raw term frequency). For example:
* query: troll $\rightarrow$ (1)
* query: troll trace $\rightarrow$ (1, 1)

**Output:**
For each given query in Part 2, you should just rank the documents retrieved by your boolean search. You only need to output the top-5 documents plus the cosine score of each of these documents. Please use the following format to present your results:

* query: relational database
* result 1:
* score: 0.1
* entity: database management system
* definition id: 656
* definition: software system used to manage databases
* result 2:
* ......
* query: garbage collection
* ......
* query: retrieval model
* ......

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!

In [15]:
# your code here
def get_vocabulary(tokens: Series):
    distinct_set = set()
    
    for index, value in tokens.items():
        distinct_set.update(value)
    
    index_dict = dict()
    idx = 0
    for val in distinct_set:
        index_dict[val] = idx
        idx += 1

    return index_dict

def get_df_dict(tokens):
    df_dict = defaultdict(int)
    for index, value in tokens.items():
        for ele in set(value):
            df_dict[ele] += 1
            
    return df_dict
    

def cal_vsm_score(tokens):
    word_dict = defaultdict(int)
    for token in tokens:
        word_dict[token] = word_dict[token] + 1
    
    vector = np.zeros(len(vocabulary))
    for key,val in word_dict.items():
        score = (1 + math.log10(val)) * (math.log10(N / df_dict[key]))
        vector[vocabulary[key]] = score
        
    return normalize(vector[:,np.newaxis], axis=0).ravel()

def cal_norm_tf_idf_vector(results, N):
    df = len(results)
    
    return results["tokens"].apply(cal_vsm_score)

def cal_cosine(query, vector):
    words = parse(Series(re.split(" ", query)), show_log=False)
    query_vector = np.zeros(len(vocabulary))
    for word in words:
        # if the query word is not in the vocabulary, ignore it
        if word[0] in vocabulary:
            query_vector[vocabulary[word[0]]] += 1
    query_vector = normalize(query_vector[:,np.newaxis], axis=0).ravel()
    
    return vector.apply(lambda v: np.sum(v * query_vector))
    

In [16]:
vocabulary = get_vocabulary(documents_df['tokens'])
df_dict = get_df_dict(documents_df['tokens'])
score_label = "vsm_score"

results1["vector"] = cal_norm_tf_idf_vector(results1, N)
results1["vsm_score"] = cal_cosine(query1, results1["vector"])
sorted_vsm_results1 = results1.sort_values(score_label, ascending=False)
print_scored_result(query1, sorted_vsm_results1, score_label, documents_df)
sorted_vsm_results1[:5]

query: relational database
result 1:
score: 0.751415
entity: relational database
definition id: 28227
definition: a database using the relational data model.

result 2:
score: 0.669222
entity: relational database
definition id: 28210
definition: a collection of related database tables

result 3:
score: 0.669222
entity: relational model
definition id: 771
definition: a database is a collection of relations or tables.

result 4:
score: 0.657223
entity: relational database
definition id: 28134
definition: a database including tables that are related to each other

result 5:
score: 0.604809
entity: relational database
definition id: 28205
definition: a database built using the relational database model



,entity,id,definition,tokens,score,vector,vsm_score
28227,relational database,28227,a database using the relational data model.\n,"[databas, use, relat, data, model]",2.720034,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.751415
28210,relational database,28210,a collection of related database tables\n,"[collect, relat, databas, tabl]",2.720034,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.669222
771,relational model,771,a database is a collection of relations or tab...,"[databas, collect, relat, tabl]",2.720034,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.669222
28134,relational database,28134,a database including tables that are related t...,"[databas, includ, tabl, relat]",2.720034,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.657223
28205,relational database,28205,a database built using the relational database...,"[databas, built, use, relat, databas, model]",3.097500,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.604809


In [17]:
results2["vector"] = cal_norm_tf_idf_vector(results2, N)
results2["vsm_score"] = cal_cosine(query2, results2["vector"])
sorted_vsm_results2 = results2.sort_values(score_label, ascending=False)
print_scored_result(query2, sorted_vsm_results2, score_label, documents_df)
sorted_vsm_results2[:5]

query: garbage collection
result 1:
score: 0.726796
entity: garbage collector
definition id: 4150
definition: the part of the operating system that performs garbage collection.

result 2:
score: 0.443649
entity: memory safety
definition id: 13115
definition: memory management handled differently such that there is garbage collection, prevent dangling pointer references.

result 3:
score: 0.431503
entity: garbage collection
definition id: 21576
definition: a class instance is explicitly created by the java code and after use it is automatically destroyed by garbage collection for memory management.

result 4:
score: 0.410964
entity: garbage collection
definition id: 21555
definition: when data is no longer referable (i.e., there are no remaining references to that data available for executable code), it is &"garbage collected&" and will be destroyed at some later point in time

result 5:
score: 0.406700
entity: garbage collection
definition id: 21553
definition: garbage collection (gc) 

,entity,id,definition,tokens,score,vector,vsm_score
4150,garbage collector,4150,the part of the operating system that performs...,"[part, oper, system, perform, garbag, collect]",4.864784,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.726796
13115,memory safety,13115,memory management handled differently such tha...,"[memori, manag, handl, differ, garbag, collect...",4.864784,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.443649
21576,garbage collection,21576,a class instance is explicitly created by the ...,"[class, instanc, explicitli, creat, java, code...",4.864784,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.431503
21555,garbage collection,21555,"when data is no longer referable (i.e., there ...","[data, longer, refer, e, remain, refer, data, ...",4.864784,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.410964
21553,garbage collection,21553,garbage collection (gc) is a form of automatic...,"[garbag, collect, gc, form, automat, memori, m...",6.530032,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.406700


In [18]:
results3["vector"] = cal_norm_tf_idf_vector(results3, N)
results3["vsm_score"] = cal_cosine(query3, results3["vector"])
sorted_vsm_results3 = results3.sort_values(score_label, ascending=False)
print_scored_result(query3, sorted_vsm_results3, score_label, documents_df)
sorted_vsm_results3[:5]

query: retrieval model
result 1:
score: 0.641719
entity: query language
definition id: 9085
definition: used to update and retrieve data that is stored in a data model

result 2:
score: 0.595900
entity: online analytical processing
definition id: 19727
definition: tools for retrieving, processing, and modelling data from the data warehouse

result 3:
score: 0.594407
entity: online analytical processing
definition id: 19731
definition: enable retrieving, processing, and modeling data from the data warehouse

result 4:
score: 0.478799
entity: online analytical processing
definition id: 19708
definition: olap - tools for retrieving, processing, and modeling data from the data warehouse

result 5:
score: 0.438052
entity: online analytical processing
definition id: 19733
definition: a set of tools that provide advanced data analysis for retrieving, processing, and modeling data from the data warehouse



,entity,id,definition,tokens,score,vector,vsm_score
9085,query language,9085,used to update and retrieve data that is store...,"[use, updat, retriev, data, store, data, model]",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.641719
19727,online analytical processing,19727,"tools for retrieving, processing, and modellin...","[tool, retriev, process, model, data, data, wa...",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.595900
19731,online analytical processing,19731,"enable retrieving, processing, and modeling da...","[enabl, retriev, process, model, data, data, w...",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.594407
19708,online analytical processing,19708,"olap - tools for retrieving, processing, and m...","[olap, tool, retriev, process, model, data, da...",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.478799
19733,online analytical processing,19733,a set of tools that provide advanced data anal...,"[set, tool, provid, advanc, data, analysi, ret...",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.438052


### C: Ranking with BM25 (20 points) 
Finally, let's try the BM25 approach for ranking. Refer to https://en.wikipedia.org/wiki/Okapi_BM25 for the specific formula. You could choose k_1 = 1.2 and b = 0.75 but feel free to try other options.

**Output:**
For each given query in Part 2, you should just rank the documents retrieved by your boolean search. You only need to output the top-5 documents plus the BM25 score of each of these documents. Please use the following format to present your results:

* query: relational database
* result 1:
* score: 0.1
* entity: database management system
* definition id: 656
* definition: software system used to manage databases
* result 2:
* ......
* query: garbage collection
* ......
* query: retrieval model
* ......

In [19]:
# your code here

def help_cal_bm25_score(terms, tokens, N, df_dict, avgdl):
    k1, b = 1.2, 0.75
    len_d = len(tokens)
    score = 0.0
    for term in terms:
        tf = tokens.count(term[0])
        idf = math.log10(N / df_dict[term[0]])
        score += idf * tf * (k1 + 1) / (tf + k1 * (1 - b + b * len_d / avgdl))
    return score

def cal_bm25(query, results, N, df_dict, avgdl):
    terms = parse(Series(re.split(" ", query)), show_log=False)
    return results["tokens"].apply(lambda tokens: help_cal_bm25_score(terms, tokens, N, df_dict, avgdl))
    

In [20]:
score_label = "bm25_score"
avgdl = documents_df["tokens"].apply(lambda tokens: len(tokens)).sum() / len(documents_df)

results1 = results1.assign(bm25_score=cal_bm25(query1, results1, N, df_dict, avgdl))
sorted_bm25_results1 = results1.sort_values(score_label, ascending=False)
print_scored_result(query1, sorted_bm25_results1, score_label, documents_df)
sorted_bm25_results1[:5]

query: relational database
result 1:
score: 4.128429
entity: relational database
definition id: 28234
definition: a database that is modeled using the relational database model a collection of related relations within which each relation has a unique name

result 2:
score: 3.847643
entity: relational database
definition id: 28205
definition: a database built using the relational database model

result 3:
score: 3.829146
entity: relational database
definition id: 28312
definition: a collection of related relations in which each relation has a unique name  operational/transactional databases

result 4:
score: 3.793218
entity: relational model
definition id: 795
definition: - database is a collection of relations - each relation has attributes and a collection of tuples

result 5:
score: 3.738525
entity: database schema
definition id: 19701
definition: set of all relation schemas in the database



,entity,id,definition,tokens,score,vector,vsm_score,bm25_score
28234,relational database,28234,a database that is modeled using the relationa...,"[databas, model, use, relat, databas, model, c...",3.980193,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.545311,4.128429
28205,relational database,28205,a database built using the relational database...,"[databas, built, use, relat, databas, model]",3.097500,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.604809,3.847643
28312,relational database,28312,a collection of related relations in which eac...,"[collect, relat, relat, relat, uniqu, name, op...",3.419553,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.493127,3.829146
795,relational model,795,- database is a collection of relations - each...,"[databas, collect, relat, relat, attribut, col...",3.161381,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.505442,3.793218
19701,database schema,19701,set of all relation schemas in the database\n,"[set, relat, schema, databas]",2.720034,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.560687,3.738525


In [21]:
results2 = results2.assign(bm25_score=cal_bm25(query2, results2, N, df_dict, avgdl))
sorted_bm25_results2 = results2.sort_values(score_label, ascending=False)
print_scored_result(query2, sorted_bm25_results2, score_label, documents_df)
sorted_bm25_results2[:5]

query: garbage collection
result 1:
score: 6.112315
entity: garbage collector
definition id: 4150
definition: the part of the operating system that performs garbage collection.

result 2:
score: 5.980504
entity: garbage collection
definition id: 21553
definition: garbage collection (gc) is a form of automatic memory management. the garbage collector, or just collector, attempts to reclaim garbage, or memory occupied by objects that are no longer in use by the program.

result 3:
score: 5.414088
entity: garbage collection
definition id: 21554
definition: there is a third phase of 2pc called garbage collection. replicas must retain records of past transactions, just in case leader fails. in practice, leader periodically tells replicas to garbage collect.

result 4:
score: 5.216599
entity: memory safety
definition id: 13115
definition: memory management handled differently such that there is garbage collection, prevent dangling pointer references.

result 5:
score: 4.700032
entity: garbag

,entity,id,definition,tokens,score,vector,vsm_score,bm25_score
4150,garbage collector,4150,the part of the operating system that performs...,"[part, oper, system, perform, garbag, collect]",4.864784,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.726796,6.112315
21553,garbage collection,21553,garbage collection (gc) is a form of automatic...,"[garbag, collect, gc, form, automat, memori, m...",6.530032,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.406700,5.980504
21554,garbage collection,21554,there is a third phase of 2pc called garbage c...,"[third, phase, 2pc, call, garbag, collect, rep...",6.329230,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.351382,5.414088
13115,memory safety,13115,memory management handled differently such tha...,"[memori, manag, handl, differ, garbag, collect...",4.864784,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.443649,5.216599
21576,garbage collection,21576,a class instance is explicitly created by the ...,"[class, instanc, explicitli, creat, java, code...",4.864784,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.431503,4.700032


In [22]:
results3 = results3.assign(bm25_score=cal_bm25(query3, results3, N, df_dict, avgdl))
sorted_bm25_results3 = results3.sort_values(score_label, ascending=False)
print_scored_result(query3, sorted_bm25_results3, score_label, documents_df)
sorted_bm25_results3[:5]

query: retrieval model
result 1:
score: 4.020224
entity: query language
definition id: 9085
definition: used to update and retrieve data that is stored in a data model

result 2:
score: 4.020224
entity: online analytical processing
definition id: 19727
definition: tools for retrieving, processing, and modelling data from the data warehouse

result 3:
score: 4.020224
entity: online analytical processing
definition id: 19731
definition: enable retrieving, processing, and modeling data from the data warehouse

result 4:
score: 3.861295
entity: online analytical processing
definition id: 19708
definition: olap - tools for retrieving, processing, and modeling data from the data warehouse

result 5:
score: 3.451909
entity: information processing
definition id: 17315
definition: taking in information, figuring out whats important, storing and retrieving processes  computer model= same software



,entity,id,definition,tokens,score,vector,vsm_score,bm25_score
9085,query language,9085,used to update and retrieve data that is store...,"[use, updat, retriev, data, store, data, model]",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.641719,4.020224
19727,online analytical processing,19727,"tools for retrieving, processing, and modellin...","[tool, retriev, process, model, data, data, wa...",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.595900,4.020224
19731,online analytical processing,19731,"enable retrieving, processing, and modeling da...","[enabl, retriev, process, model, data, data, w...",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.594407,4.020224
19708,online analytical processing,19708,"olap - tools for retrieving, processing, and m...","[olap, tool, retriev, process, model, data, da...",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.478799,3.861295
17315,information processing,17315,"taking in information, figuring out whats impo...","[take, inform, figur, what, import, store, ret...",3.337042,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.361087,3.451909


### Discussion
Briefly discuss the differences you see between the three methods. Is there one you prefer?

#### My Answer

For TF-IDF, it would prefer larger documents, since the larger documents have more tokens and more tokens will have a larger possiblity for increasing the term frequency.

For Vector Space Model, since we use simple weights to represent the query vector and our queries all consist of two single words, it will tend to select the documents in which each query term only occurs once as the top ranked documents.

For BM25, it just combines the two above approach, so it will select the documents with the medium length. And I prefer this method most.


## Bonus: Evaluation (10 points)
Rather than just compare methods by pure observation, there are several metrics to evaluate the performance of an IR engine: Precision, Recall, MAP, NDCG, HitRate and so on. These all require a ground truth set of queries and documents with a notion of **relevance**. These ground truth judgments can be expensive to obtain, so we are cutting corners here and treating a flashcard's front and back as a "relevant" query-document pair.

That is, if a document (definition) in your top-5 results is from the back of query's (entity's) flashcard, this document is regarded as relevant to the query (entity). This document is also called a hit in IR. Based on the ground-truth, you could calculate the metrics for the three ranking methods and provide the results like these:

* metric: Precision@5
* TF-IDF - score1
* Vector Space Model with TF-IDF - score2
* BM25 - score3

You could pick any of the reasonable metrics.

In [23]:
# your code here
# calculate the precision for each query
def cal_precision(query, sorted_results):
    return sorted_results["entity"][:5].apply(lambda entity: 1 if entity == query else 0).sum() / 5


In [24]:
print("metric: Precision@5")

score_tf_idf = (cal_precision(query1, sorted_tf_idf_results1) + cal_precision(query2, sorted_tf_idf_results2) + cal_precision(query3, sorted_tf_idf_results3)) / 3
print("TF-IDF - %f" % score_tf_idf)

score_vsm = (cal_precision(query1, sorted_vsm_results1) + cal_precision(query2, sorted_vsm_results2) + cal_precision(query3, sorted_vsm_results3)) / 3
print("Vector Space Model with TF-IDF - %f" % score_vsm)

score_bm25 = (cal_precision(query1, sorted_bm25_results1) + cal_precision(query2, sorted_bm25_results2) + cal_precision(query3, sorted_bm25_results3)) / 3
print("BM25 - %f" % score_bm25)

metric: Precision@5
TF-IDF - 0.400000
Vector Space Model with TF-IDF - 0.466667
BM25 - 0.400000


#### My Answer

The average precisions of these three methods are 0.4, 0.47, 0.4 resepectively. 

Thus, based on the results above, Vector Space Model has the better performance over the other two approaches.

# Collaboration Declarations

** You should fill out your collaboration declarations here.**

**Reminder:** You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by filling out the Collaboration Declarations at the bottom of this notebook.

Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.